In [0]:
# Import required libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [0]:
# Show available cpu cores
import multiprocessing
n_jobs = multiprocessing.cpu_count()-1
n_jobs

3

In [0]:
# Read HBB and HEXA file
df5 = pd.read_excel("dataset.xlsx")
df5 = df5.drop('Spacer sequence', axis=1)
df5.rename(columns={"3' extension":"3extension"}, inplace=True)
df5['Mean of correct edit'] = df5.iloc[:,1:3].mean(axis=1)

In [0]:
df5 = df5.drop(['% correct edit (w/o indels), rep 1', '% correct edit (w/o indels), rep 2', '% correct edit (w/o indels), rep 3'], axis=1)

## **Same length sequences method**



In [0]:
df4 = pd.read_excel("mtrx.xlsx")

In [0]:
df4['PBS length'] = df5['PBS length']
df4['RT template length'] = df5['RT template length']
df4['Mean of correct edit'] = df5['Mean of correct edit']

In [0]:
df4 = df4.drop(['Unnamed: 0'], axis=1)

In [0]:
# Replace a, c, g, t with 0.25, 0.50, 0.75, 1.00 repectively, this is equal to ordinal encoding
df4.replace(['a', 'c', 'g', 't'], [0.25, 0.50, 0.75, 1.00], inplace=True)

In [0]:
vectors_40mtrx = df4

 Set features as X and target variable as y

In [0]:
# Features
X = vectors_40mtrx.drop('Mean of correct edit', axis=1)

In [0]:
# Target variable
y = df5['Mean of correct edit']

In [0]:
# Splitting the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

Random Forest Regressor

In [0]:
# Import the Random Forest Regressor model
from sklearn.ensemble import RandomForestRegressor

In [0]:
print(X_train.shape)
print(X_test.shape)

(39, 42)
(17, 42)


In [0]:
print(y_train.shape)
print(y_test.shape)

(39,)
(17,)


In [0]:
# Grid search the Random Forest Regressor
from sklearn.model_selection import GridSearchCV

# Setting the static parameters
rfr = RandomForestRegressor(bootstrap=True, random_state=42, n_jobs=3)

param_grid = dict(n_estimators=[10, 25, 50, 100, 1000],
                  max_depth=[5, 10, 20, 30],
                  min_samples_leaf=[1,2,4])

grid = GridSearchCV(rfr, param_grid, cv=10,
                    scoring='neg_mean_squared_error')
grid.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=3,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs=N

In [0]:
print("grid.cv_results_ {}".format(grid.cv_results_))

In [0]:
print("grid.best_params_ {}".format(grid.best_params_))
print("grid.best_estimator_ {}".format(grid.best_estimator_)) 

grid.best_params_ {'max_depth': 5, 'min_samples_leaf': 1, 'n_estimators': 100}
grid.best_estimator_ RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=5, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=3, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)


In [0]:
# Instantiate model with grid search parameters
rf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=5, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=3, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [0]:
# Train the model on training data
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=5, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=3, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [0]:
# Use the forest's predict method on the test data
predictions = rf.predict(X)

In [0]:
# Place predictions in the dataframe
vectors_40mtrx['Predictions'] = pd.DataFrame(data=predictions)

In [0]:
results = vectors_40mtrx[['Mean of correct edit', 'Predictions']] 

In [0]:
# Print the actual values and the model's predictions
results

,Mean of correct edit,Predictions
0,40.982414,26.237241
1,32.843033,28.471201
2,36.210669,33.090670
3,46.204905,30.636344
4,42.342045,36.156731
5,44.501013,34.743312
6,38.488102,23.445953
7,29.187135,29.207399
8,29.959730,34.035801
9,40.690499,34.212310


In [0]:
import sklearn.metrics as metrics

# Calculate MAE, MSE, RMSE
print(metrics.mean_absolute_error(vectors_40mtrx['Mean of correct edit'],predictions))
print(metrics.mean_squared_error(vectors_40mtrx['Mean of correct edit'], predictions))
print(np.sqrt(metrics.mean_squared_error(vectors_40mtrx['Mean of correct edit'], predictions)))

4.73108756508952
37.979343797720176
6.162738335976969


Gradient Boosting Regressor

In [0]:
# Import the Gradient Boosting Regressor model
from sklearn.ensemble import GradientBoostingRegressor

In [0]:
#Random Search Gradient Boosting Regressor
from sklearn.model_selection import GridSearchCV

param_grid = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
'max_depth': [4, 6],
'min_samples_leaf': [3, 5, 9, 17],
'max_features': [1.0, 0.3, 0.1]}

est = GradientBoostingRegressor(n_estimators=3000)
gs_cv = GridSearchCV(est, param_grid, n_jobs=4).fit(X_train, y_train)

# Best hyperparameter setting
gs_cv.best_params_

{'learning_rate': 0.01,
 'max_depth': 4,
 'max_features': 0.1,
 'min_samples_leaf': 9}

In [0]:
# Parameters from grid search
gbr = GradientBoostingRegressor(n_estimators=3000, max_depth=4, learning_rate=0.01,
                                loss='huber', max_features=0.1,
                                min_samples_leaf=9)

In [0]:
gbr.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='huber',
                          max_depth=4, max_features=0.1, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=9, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
# Use the Gradient Boosting Regressor predict method on the test data
predictions2 = gbr.predict(X)

In [0]:
# Place predictions in the dataframe
vectors_40mtrx['Predictions 2'] = pd.DataFrame(data=predictions2)

In [0]:
results2 = vectors_40mtrx[['Mean of correct edit', 'Predictions 2']] 

In [0]:
# Print the actual values and the model's predictions
results2

,Mean of correct edit,Predictions 2
0,40.982414,29.565791
1,32.843033,32.738961
2,36.210669,37.805063
3,46.204905,35.533021
4,42.342045,39.412034
5,44.501013,41.216907
6,38.488102,30.553989
7,29.187135,30.367173
8,29.959730,37.958706
9,40.690499,37.950018


In [0]:
import sklearn.metrics as metrics

# Calculate MAE, MSE, RMSE
print(metrics.mean_absolute_error(vectors_40mtrx['Mean of correct edit'], predictions2))
print(metrics.mean_squared_error(vectors_40mtrx['Mean of correct edit'], predictions2))
print(np.sqrt(metrics.mean_squared_error(vectors_40mtrx['Mean of correct edit'], predictions2)))

3.431383267098005
25.567939753671965
5.056475032438306


## **Fillna = 0 method**

In [0]:
# Split each sequence to columnns and convert base pairs to lowercase
df3 = df5['3extension'].apply(lambda x: [z for y in x for z in y.strip("'").lower().split()])

In [0]:
df3 = df3.apply(pd.Series)

In [0]:
# Replace a,c,g,t with 0.25, 0.50, 0.75, 1.00 repectively, this is equal to ordinal encoding
df3.replace(['a', 'c', 'g', 't'], [0.25, 0.50, 0.75, 1.00], inplace=True)

In [0]:
# Replace NaN values with 0
df3 = df3.fillna(0)

In [0]:
df3['PBS length'] = df5['PBS length']
df3['RT template length'] = df5['RT template length']
df3['Mean of correct edit'] = df5['Mean of correct edit']

In [0]:
df3.sort_values(['RT template length'], ascending=[False])

In [0]:
# Features
X = df3.drop('Mean of correct edit', axis=1)

In [0]:
# Target variable
y = df5['Mean of correct edit']

In [0]:
# Import the Random Forest Regressor model
from sklearn.ensemble import RandomForestRegressor

In [0]:
# Splitting the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [0]:
print(X_train.shape)
print(X_test.shape)

(39, 40)
(17, 40)


In [0]:
print(y_train.shape)
print(y_test.shape)

(39,)
(17,)


In [0]:
# Grid search the Random Forest Regressor
from sklearn.model_selection import GridSearchCV

# Setting the static parameters
rfr = RandomForestRegressor(bootstrap=True, random_state=42, n_jobs=4)

param_grid = dict(n_estimators=[10, 25, 50, 100, 1000],
                  max_depth=[5, 10, 20, 30],
                  min_samples_leaf=[1,2,4])

grid = GridSearchCV(rfr, param_grid, cv=10,
                    scoring='neg_mean_squared_error')
grid.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=4,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jobs=N

In [0]:
print("grid.cv_results_ {}".format(grid.cv_results_))

In [0]:
print("-grid.best_score_ {}".format(-grid.best_score_))
print("grid.best_params_ {}".format(grid.best_params_))
print("grid.best_estimator_ {}".format(grid.best_estimator_)) 

-grid.best_score_ 98.10587088702927
grid.best_params_ {'max_depth': 20, 'min_samples_leaf': 1, 'n_estimators': 1000}
grid.best_estimator_ RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=4, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)


In [0]:
# Instantiate model with grid search parameters
rf = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=4, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [0]:
# Train the model on training data
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=20, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=4, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [0]:
# Use the forest's predict method on the test data
predictions = rf.predict(X)

In [0]:
# Place predictions in the dataframe
df3['Predictions'] = pd.DataFrame(data=predictions)

In [0]:
# Print the actual values and the model's predictions
df3[['Mean of correct edit', 'Predictions']] 

,Mean of correct edit,Predictions
0,40.982414,21.381649
1,32.843033,28.738437
2,36.210669,29.938973
3,46.204905,31.370687
4,42.342045,36.318928
5,44.501013,31.462871
6,38.488102,10.144545
7,29.187135,28.063951
8,29.959730,30.067672
9,40.690499,31.825738


In [0]:
import sklearn.metrics as metrics

# Calculate MAE, MSE, RMSE
print(metrics.mean_absolute_error(df3['Mean of correct edit'],predictions))
print(metrics.mean_squared_error(df3['Mean of correct edit'], predictions))
print(np.sqrt(metrics.mean_squared_error(df3['Mean of correct edit'], predictions)))

4.940090990581845
55.647902350758145
7.4597521641645805


Gradient Boosting Regressor

In [0]:
# Import the Gradient Boosting Regressor model
from sklearn.ensemble import GradientBoostingRegressor

In [0]:
#Random Search Gradient Boosting Regressor
from sklearn.model_selection import GridSearchCV

param_grid = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
'max_depth': [4, 6],
'min_samples_leaf': [3, 5, 9, 17],
'max_features': [1.0, 0.3, 0.1]}

est = GradientBoostingRegressor(n_estimators=3000)
gs_cv = GridSearchCV(est, param_grid, n_jobs=4).fit(X_train, y_train)

# Best hyperparameter setting
gs_cv.best_params_

{'learning_rate': 0.01,
 'max_depth': 4,
 'max_features': 1.0,
 'min_samples_leaf': 3}

In [0]:
# Parameters from grid search
gbr = GradientBoostingRegressor(n_estimators=3000, max_depth=4, learning_rate=0.01,
                                loss='huber', max_features=1.0,
                                min_samples_leaf=3)

In [0]:
gbr.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='huber',
                          max_depth=4, max_features=1.0, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=3, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
# Use the Gradient Boosting Regressor predict method on the test data
predictions2 = gbr.predict(X)

In [0]:
# Place predictions in the dataframe
df3['Predictions 2'] = pd.DataFrame(data=predictions2)

In [0]:
# Print the actual values and the model's predictions
df3[['Mean of correct edit', 'Predictions 2']] 

,Mean of correct edit,Predictions 2
0,40.982414,13.124828
1,32.843033,32.813572
2,36.210669,36.209424
3,46.204905,33.847808
4,42.342045,42.319322
5,44.501013,36.720060
6,38.488102,3.957600
7,29.187135,29.207630
8,29.959730,34.599080
9,40.690499,40.688284


In [0]:
import sklearn.metrics as metrics

# Calculate MAE, MSE, RMSE
print(metrics.mean_absolute_error(df3['Mean of correct edit'], predictions2))
print(metrics.mean_squared_error(df3['Mean of correct edit'], predictions2))
print(np.sqrt(metrics.mean_squared_error(df3['Mean of correct edit'], predictions2)))

3.2025175823174203
61.25423908037336
7.826508741474282
